### import dependencies

In [4]:
##### work in this repository was inspired by 
##### https://github.com/Kazuhito00/hand-gesture-recognition-using-mediapipe/blob/main/README_EN.md
##### https://www.youtube.com/watch?v=a99p_fAr6e4


### import dependencies

In [5]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd


# this is just to make sure that the results are reproducible to anyone that runs the code lol.
RANDOM_SEED = 42

#### data formatting
https://devqa.io/python-convert-csv-file-to-list/

In [6]:
import glob
labels = {0: "no_yawn", 1: "yawn"}

Y_train = [] #1d array, (big_number, 1)
X_train = [] #2d array, (big_number, 42)
def data_append_train(data, X_train = X_train, Y_train = Y_train):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/train"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_train.append([float(num) for num in row])
                Y_train.append(label)
    
    print(X_train[-1])
    print("training set:")
    print(len(X_train))
    print(len(Y_train))    

Y_test = []
X_test = []
def data_append_test(data, X_test = X_test, Y_test = Y_test):
  #formatting the labels, although I am not so sure what is required...
  data1 = "data/test"
  for label in labels: 
    name = labels[label]
    path = f"{data1}/{name}/*.csv"
    print(name)

    # getting the directory
    files = glob.glob(path)
    print(files)
    print(len(files))

    # now concatenating the content of the files.
    for f in files:
        with open(f, 'r') as file:
            csv_reader = csv.reader(file)
        
            i = 0
            for row in csv_reader:
                #skip the header line
                if i == 0: 
                    i += 1
                    continue
                
                X_test.append([float(num) for num in row])
                Y_test.append(label)
    
    print(X_test[-1])
    print("training set:")
    print(len(X_test))
    print(len(Y_test))    
#i'll only use the train test.
data = "data"
data_append_train(data)
data_append_test(data)




no_yawn
['data/train/no_yawn\\eye.csv']
1
[335.7875442504883, 307.1158504486084, -47.087249755859375, 336.71512603759766, 296.57092094421387, -51.64748668670654, 336.2636947631836, 280.96996307373047, -49.989943504333496, 333.3477020263672, 241.02779388427734, -31.407363414764404, 332.73876190185547, 212.79132843017578, -29.069066047668457, 327.1074676513672, 345.0590229034424, -8.09687077999115, 327.3999786376953, 306.43741607666016, -47.44572639465332, 272.0389938354492, 340.79870223999023, 4.277186393737793, 213.45247268676758, 321.92012786865234, 75.54635047912598, 306.8148422241211, 304.96450424194336, -23.216757774353027, 298.4514808654785, 302.03410148620605, -22.908165454864502, 290.2219581604004, 345.4917812347412, -0.9893196076154709, 317.0164680480957, 344.7213363647461, -9.153054356575012, 299.5151901245117, 343.9896011352539, -4.636331796646118, 298.1235122680664, 345.52124977111816, -5.400487184524536, 202.47108459472656, 267.89669036865234, 77.93771266937256, 232.5141143

### since I already have the training dataset split, I just need to train!

In [7]:
# X_df_train = pd.DataFrame(X_train)
# Y_df_train = pd.DataFrame(Y_train)

# X_df_test = pd.DataFrame(X_test)
# Y_df_test = pd.DataFrame(Y_test)


X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
# 
X_test = np.asarray(X_test)
Y_test = np.asarray(Y_test)

from sklearn.utils import shuffle

# the random_state is so that both shuffle perform the same.
X_train,Y_train = shuffle(X_train, Y_train, random_state=0)
X_test,Y_test = shuffle(X_test, Y_test, random_state=0)
# X_train.shape, Y_train.shape
X_train, Y_train


(array([[353.44478607, 276.75739288, -52.59527206, ..., 366.1561203 ,
         275.91233253, -45.3466177 ],
        [337.43492126, 301.17736816, -55.10536671, ..., 351.42662048,
         297.77366638, -45.172925  ],
        [317.25000381, 314.96389389, -41.72193527, ..., 329.83459473,
         312.71109581, -37.76688814],
        ...,
        [381.67285919, 291.85976028, -44.33598995, ..., 390.51780701,
         288.99355888, -31.44557953],
        [395.69858551, 326.77339554, -33.11842203, ..., 401.57093048,
         327.70305634, -23.92225981],
        [344.98332977, 302.50336647, -48.38704586, ..., 357.84194946,
         299.94438171, -40.4253149 ]]),
 array([1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1,
        0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
        0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 

In [8]:
NUM_CLASSES = 2 
# model_save_path = "model/"
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "model/yawn_model/model.{epoch:02d}-{val_accuracy:.2f}",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    #everytime the accuracy gets better, it saves
    save_freq='epoch',
)


#defining the model 

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import keras

In [10]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)


Shape of X_train: (990, 174)
Shape of X_test: (700, 174)


In [11]:

model = Sequential([
    Dense(units=32, activation='relu', input_shape=(174,)),
    Dense(units=64, activation='relu',
        kernel_regularizer=keras.regularizers.l1_l2(0.05)),
    Dense(units=256, activation='relu'),
    # Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=16, activation='relu', 
        kernel_regularizer=keras.regularizers.l1_l2(0.01)),
    # Dropout(0.2),
    Dense(units=NUM_CLASSES, activation='softmax')
])

In [12]:
#cost function
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model.compile(loss=SparseCategoricalCrossentropy(), optimizer=Adam(), metrics=["accuracy"])

In [13]:
#fitting
history = model.fit(X_train,Y_train, epochs=150, batch_size=32, validation_data=(X_test,Y_test), callbacks=[checkpoint])

Epoch 1/150
31/31 [==============================] - 7s 170ms/step - loss: 19.0029 - accuracy: 0.6131 - val_loss: 13.1956 - val_accuracy: 0.5443
Epoch 2/150
31/31 [==============================] - 5s 154ms/step - loss: 11.9540 - accuracy: 0.7030 - val_loss: 11.0032 - val_accuracy: 0.7029
Epoch 3/150
31/31 [==============================] - 0s 10ms/step - loss: 9.8829 - accuracy: 0.7879 - val_loss: 9.3592 - val_accuracy: 0.5614
Epoch 4/150
31/31 [==============================] - 0s 10ms/step - loss: 8.1583 - accuracy: 0.8404 - val_loss: 7.6778 - val_accuracy: 0.6443
Epoch 5/150
31/31 [==============================] - 6s 193ms/step - loss: 6.6804 - accuracy: 0.8929 - val_loss: 6.1796 - val_accuracy: 0.8571
Epoch 6/150
31/31 [==============================] - 0s 12ms/step - loss: 5.5732 - accuracy: 0.9111 - val_loss: 5.3030 - val_accuracy: 0.8400
Epoch 7/150
31/31 [==============================] - 5s 152ms/step - loss: 4.6742 - accuracy: 0.9727 - val_loss: 4.4637 - val_accuracy: 0.951